In [2]:
# This script is based on:
# https://www.tensorflow.org/get_started/mnist/pros

import sys
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import graph_util as gu
from tensorflow.python.framework.graph_util import remove_training_nodes
from tensorflow.tools.graph_transforms import TransformGraph

In [3]:
mnist = input_data.read_data_sets("mnist_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


#### training set

In [27]:
images_train = mnist.train.images
labels_train = mnist.train.labels
cls_train = np.matmul(labels_train, np.arange(10))
cls_train.shape

(55000,)

In [28]:
order = cls_train.argsort()
cls_train_sorted = cls_train[order]
images_train_sorted = images_train[order]
labels_train_sorted = labels_train[order]

In [29]:
print("The distribution of digits is uneven. ")
pd.Series(cls_train_sorted).value_counts()

The distribution of digits is uneven. 


1.0    6179
7.0    5715
3.0    5638
2.0    5470
9.0    5454
0.0    5444
6.0    5417
8.0    5389
4.0    5307
5.0    4987
dtype: int64

In [30]:
cls_df = pd.Series(cls_train_sorted)
cls_df = cls_df.loc[cls_df < 5]
count_0to4 = cls_df.shape[0]
print("{} numbers from 0 to 4".format(count_0to4))
cls_df.value_counts()

28038 numbers from 0 to 4


1.0    6179
3.0    5638
2.0    5470
0.0    5444
4.0    5307
dtype: int64

In [31]:
images_train_sorted_0to4 = images_train_sorted[0:count_0to4, :]
labels_train_sorted_0to4 = labels_train_sorted[0:count_0to4, :]
cls_train_sorted_0to4 = cls_train_sorted[0:count_0to4]
print(images_train_sorted_0to4.shape)
print(labels_train_sorted_0to4.shape)
print(cls_train_sorted_0to4.shape)

(28038, 784)
(28038, 10)
(28038,)


In [34]:
print("Shuffle data")
np.random.seed(101)
np.random.shuffle(images_train_sorted_0to4)
np.random.seed(101)
np.random.shuffle(labels_train_sorted_0to4)
np.random.seed(101)
np.random.shuffle(cls_train_sorted_0to4)

Shuffle data


In [35]:
print("Check the three arrays match:")
print(labels_train_sorted_0to4[[10, 101, 10001, 20000], :])
print(cls_train_sorted_0to4[[10, 101, 10001, 20000]])

Check the three arrays match:
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[4. 3. 3. 2.]


#### validation set

In [37]:
images_val = mnist.validation.images
labels_val = mnist.validation.labels
cls_val = np.matmul(labels_val, np.arange(10))
cls_val.shape

(5000,)

In [38]:
order = cls_val.argsort()
cls_val_sorted = cls_val[order]
images_val_sorted = images_val[order]
labels_val_sorted = labels_val[order]

In [39]:
print("The distribution of digits is uneven. ")
pd.Series(cls_val_sorted).value_counts()

The distribution of digits is uneven. 


1.0    563
7.0    550
4.0    535
6.0    501
9.0    495
3.0    493
2.0    488
0.0    479
8.0    462
5.0    434
dtype: int64

In [41]:
cls_val_df = pd.Series(cls_val_sorted)
cls_val_df = cls_val_df.loc[cls_val_df < 5]
count_val_0to4 = cls_val_df.shape[0]
print("{} numbers from 0 to 4".format(count_val_0to4))
cls_val_df.value_counts()

2558 numbers from 0 to 4


1.0    563
4.0    535
3.0    493
2.0    488
0.0    479
dtype: int64

In [42]:
images_val_sorted_0to4 = images_val_sorted[0:count_val_0to4, :]
labels_val_sorted_0to4 = labels_val_sorted[0:count_val_0to4, :]
cls_val_sorted_0to4 = cls_val_sorted[0:count_val_0to4]
print(images_val_sorted_0to4.shape)
print(labels_val_sorted_0to4.shape)
print(cls_val_sorted_0to4.shape)

(2558, 784)
(2558, 10)
(2558,)


In [43]:
print("Shuffle data")
np.random.seed(101)
np.random.shuffle(images_val_sorted_0to4)
np.random.seed(101)
np.random.shuffle(labels_val_sorted_0to4)
np.random.seed(101)
np.random.shuffle(cls_val_sorted_0to4)

Shuffle data


In [44]:
print("Check the three arrays match:")
print(labels_val_sorted_0to4[[2, 331, 445, 2000], :])
print(cls_val_sorted_0to4[[2, 331, 445, 2000]])

Check the three arrays match:
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[4. 4. 1. 2.]


### model

In [45]:
def deepnn(x):
    W_fc1 = weight_variable([784, 128], name='W_fc1')
    b_fc1 = bias_variable([128], name='b_fc1')
    a_fc1 = tf.add(tf.matmul(x, W_fc1), b_fc1, name="zscore")
    h_fc1 = tf.nn.relu(a_fc1)
    
    W_fc2 = weight_variable([128, 64], name='W_fc2')
    b_fc2 = bias_variable([64], name='b_fc2')
    a_fc2 = tf.add(tf.matmul(h_fc1, W_fc2), b_fc2, name="zscore")
    h_fc2 = tf.nn.relu(a_fc2)
    
    W_fc3 = weight_variable([64, 10], name='W_fc3')
    b_fc3 = bias_variable([10], name='b_fc3')
    logits = tf.add(tf.matmul(h_fc2, W_fc3), b_fc3, name="logits")
    y_pred = tf.argmax(logits, 1, name='y_pred')

    return y_pred, logits


def weight_variable(shape, name):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name)


def bias_variable(shape, name):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name)

In [47]:
# Reset default graph
tf.reset_default_graph()

# Create the model
x = tf.placeholder(tf.float32, [None, 784], name="x")

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10], name="y")

# Build the graph for the deep net
y_pred, logits = deepnn(x)

with tf.name_scope("Loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, 
                                                               logits=logits)
    loss = tf.reduce_mean(cross_entropy, name="cross_entropy_loss")
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss, name="train_step")
  
# Here we specify the output as "Prediction/y_pred", this will be important later
with tf.name_scope("Prediction"): 
    correct_prediction = tf.equal(y_pred, 
                                  tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

In [55]:
batch_size = 50
n_epochs = 100
n_batches = int(images_train_sorted_0to4.shape[0] / batch_size)
print(n_batches)

560


In [49]:
sess = tf.Session()
# Initialize the variables (i.e. assign their default value)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

train_loss, train_accuracy = sess.run([loss, accuracy], feed_dict={x: images_train_sorted_0to4, 
                                                                   y_: labels_train_sorted_0to4})
print('Epoch %d, training loss: %g, training accuracy: %g' % (0, train_loss, train_accuracy))
val_loss, val_accuracy = sess.run([loss, accuracy], feed_dict={x: images_val_sorted_0to4,
                                                               y_: labels_val_sorted_0to4})
print('Epoch %d, validation loss: %g, validation accuracy %g' % (0, val_loss, val_accuracy))

Epoch 0, training loss: 2.5812, training accuracy: 0.126828
Epoch 0, validation loss: 2.58384, validation accuracy 0.135262


In [57]:
for i in range(n_epochs):
    batch_start = 0
    batch_end = batch_start + batch_size
    
    for j in range(n_batches):
        if j == n_batches - 1:
            batch_images = images_train_sorted_0to4[batch_start:, :]
            batch_labels = labels_train_sorted_0to4[batch_start:, :]
        else:
            batch_images = images_train_sorted_0to4[batch_start:batch_end, :]
            batch_labels = labels_train_sorted_0to4[batch_start:batch_end, :]
        batch_start += batch_size
        batch_end += batch_size
        
        sess.run(train_step, feed_dict={x: batch_images, y_: batch_labels})
        
        
    if i % 10 == 0:
        train_loss, train_accuracy = sess.run([loss, accuracy], feed_dict={x: images_train_sorted_0to4, 
                                                                           y_: labels_train_sorted_0to4})
        print('Epoch %d, training loss: %g, training accuracy: %g' % (i, train_loss, train_accuracy))
        val_loss, val_accuracy = sess.run([loss, accuracy], feed_dict={x: images_val_sorted_0to4,
                                                                       y_: labels_val_sorted_0to4})
        print('Epoch %d, validation loss: %g, validation accuracy %g' % (i, val_loss, val_accuracy))

Epoch 0, training loss: 0.172996, training accuracy: 0.957379
Epoch 0, validation loss: 0.171218, validation accuracy 0.961689
Epoch 10, training loss: 0.0385939, training accuracy: 0.989372
Epoch 10, validation loss: 0.0499171, validation accuracy 0.983972
Epoch 20, training loss: 0.0149445, training accuracy: 0.996469
Epoch 20, validation loss: 0.0377949, validation accuracy 0.98749
Epoch 30, training loss: 0.00547536, training accuracy: 0.999215
Epoch 30, validation loss: 0.0383306, validation accuracy 0.987881
Epoch 40, training loss: 0.001544, training accuracy: 0.999857
Epoch 40, validation loss: 0.0419834, validation accuracy 0.989054
Epoch 50, training loss: 0.000368855, training accuracy: 1
Epoch 50, validation loss: 0.0503327, validation accuracy 0.989054
Epoch 60, training loss: 8.60282e-05, training accuracy: 1
Epoch 60, validation loss: 0.0598857, validation accuracy 0.990618
Epoch 70, training loss: 2.14553e-05, training accuracy: 1
Epoch 70, validation loss: 0.0709655, v

In [58]:
print('test accuracy %g' % sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

test accuracy 0.51


In [59]:
saver.save(sess, "./chkps/mnist_model_0to4")
out_nodes = [y_pred.op.name]
print(out_nodes)

Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'W_fc1' has type str, but expected one of: int, long, bool
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'W_fc1' has type str, but expected one of: int, long, bool
['y_pred']


In [60]:
sub_graph_def = remove_training_nodes(sess.graph_def)

In [61]:
sub_graph_def = gu.convert_variables_to_constants(sess, sub_graph_def, out_nodes)

INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.


In [62]:
graph_path = tf.train.write_graph(sub_graph_def,
                                  "./mnist_model_0to4",
                                  "deep_mlp.pb",
                                  as_text=False)

print('written graph to: %s' % graph_path)

written graph to: ./mnist_model_0to4/deep_mlp.pb


###

In [64]:
sess.run(y_pred, feed_dict={x: mnist.test.images[[3, 2, 1, 18, 4, 15, 11, 0, 61, 7], :]})

array([0, 1, 2, 3, 4, 3, 0, 3, 2, 4])

In [65]:
mnist.test.labels[[3, 2, 1, 18, 4, 15, 11, 0, 61, 7], :]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [68]:
images = mnist.test.images[[3, 2, 1, 18, 4, 15, 11, 0, 61, 7]]

In [69]:
np.savetxt("./images_0to9.csv", images, delimiter=",")

In [71]:
pd.DataFrame(images).to_csv("./images_0to9.csv")

In [72]:
images.shape

(10, 784)

In [92]:
with open("./input_data.h", "w") as hfile:
    hfile.write("const float input_data [%s][%s] = {\n" % (images.shape[0], images.shape[1]))
    for row in images:
        hfile.write("{")
        row_str = ", ".join([str(elem) for elem in row])
        hfile.write(row_str)
        hfile.write("},\n")
    hfile.write("};")